In [1]:
import random
import math
from IPython.display import display
import pandas as pd
import time
import numpy as np


In [2]:
def original_board():
    ttt_tictac_board = {
        1: ' ', 2: ' ', 3: ' ',
        4: ' ', 5: ' ', 6: ' ',
        7: ' ', 8: ' ', 9: ' '
    }
    SI_Agent_Letter = 'X'
    MinMax_Letter = 'O'
    return ttt_tictac_board, SI_Agent_Letter, MinMax_Letter

def show_board(tictac_board):
    print("\n")
    for row in range(3):
        for col in range(3):
            cell = row * 3 + col + 1
            print(tictac_board[cell], end="")
            if col < 2:
                print(" | ", end="")
        print()
        if row < 2:
            print("---------")
    print()

def first_turn():
    choices = [1, 2]  # Assuming 1 for one player and 2 for another player
    return random.choice(choices)

def val_turn(tictac_board, move):
    if move in tictac_board:
        return tictac_board[move] == ' '
    else:
        return False  # This handles the case where the move is not a valid key in the tictac_board dictionary


def val_draw(tictac_board):
    return ' ' not in tictac_board.values() and not val_success(tictac_board)

def val_success(tictac_board):
    win_combinations = [
        (1, 2, 3), (4, 5, 6), (7, 8, 9),
        (1, 4, 7), (2, 5, 8), (3, 6, 9),
        (1, 5, 9), (7, 5, 3)
    ]

    for combo in win_combinations:
        if tictac_board[combo[0]] == tictac_board[combo[1]] == tictac_board[combo[2]] != ' ':
            return True

    return False

def val_success_for_letter(tictac_board, mark):
    winning_positions = [
        (1, 2, 3), (4, 5, 6), (7, 8, 9),
        (1, 4, 7), (2, 5, 8), (3, 6, 9),
        (1, 5, 9), (7, 5, 3)
    ]

    for pos in winning_positions:
        if all(tictac_board[i] == mark for i in pos):
            return True
    return False

def get_random_turn(tictac_board):
    position = random.randint(1, 9)
    while not val_turn(tictac_board, position):
        position = random.randint(1, 9)
    return position



def play_ttt_with_alpha_beta_pruning(tictac_board, SIAgent_plays_first, SI_Agent_Letter, MinMax_Letter):
    while True:
        if SIAgent_plays_first:
            tictac_board = si_agent_move(tictac_board, SI_Agent_Letter)
            if val_success_for_letter(tictac_board, SI_Agent_Letter):
                return "SIAgentWon"
            if val_draw(tictac_board):
                return "Draw"
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, MinMax_Letter,SI_Agent_Letter)
            if val_success_for_letter(tictac_board, MinMax_Letter):
                return "MinMaxWon"
        else:
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, MinMax_Letter,SI_Agent_Letter)
            if val_success_for_letter(tictac_board, MinMax_Letter):
                return "MinMaxWon"
            tictac_board = si_agent_move(tictac_board, SI_Agent_Letter)
            if val_success_for_letter(tictac_board, SI_Agent_Letter):
                return "SIAgentWon"
            if val_draw(tictac_board):
                return "Draw"


def si_agent_move(tictac_board, agent_letter):
    for possible_position in tictac_board.keys():
        if tictac_board[possible_position] == ' ':
            tictac_board[possible_position] = agent_letter
            
            if val_success(tictac_board):
                tictac_board[possible_position] = ' '
                position = possible_position
                break
                
            elif val_draw(tictac_board):
                tictac_board[possible_position] = ' '
                position = possible_position
                break
                
            else:
                tictac_board[possible_position] = ' '
                position = get_random_turn(tictac_board)

        if 'position' in locals():  # Check if position has been set
            tictac_board[position] = agent_letter
            return tictac_board

    return tictac_board  # In case no move has been made, return the tictac_board as is.

def min_max_move_with_alpha_beta_pruning(tictac_board, player_letter, opponent_letter, max_depth):
    optimised_score = -math.inf
    optimised_position = get_random_turn(tictac_board)

    for possible_position in tictac_board.keys():
        if tictac_board[possible_position] == ' ':
            tictac_board[possible_position] = player_letter
            current_score = eval_minmax_with_alpha_beta_pruning(tictac_board, False, -math.inf, math.inf, player_letter, opponent_letter, val_success_for_letter, val_draw, 0, max_depth)  # Adjusted function call
            #current_score = eval_minmax_with_alpha_beta_pruning(tictac_board, False, -math.inf, math.inf, player_letter, opponent_letter)
            tictac_board[possible_position] = ' '

            if current_score > optimised_score:
                optimised_score = current_score
                optimised_position = possible_position

    if optimised_position is not None:
        tictac_board[optimised_position] = player_letter
    return tictac_board



def eval_minmax_with_alpha_beta_pruning(tictac_board, is_minmax_move, alpha, beta, minmax_letter, si_agent_letter, val_success_for_letter, val_draw,depth, max_depth):
    if val_success_for_letter(tictac_board, minmax_letter):
        return 1
    elif val_success_for_letter(tictac_board, si_agent_letter):
        return -1
    elif val_draw(tictac_board):
        return 0

    if is_minmax_move:
        optimised_score = -math.inf
        for possible_position in tictac_board.keys():
            if tictac_board[possible_position] == ' ':
                tictac_board[possible_position] = minmax_letter
                current_score = eval_minmax_with_alpha_beta_pruning(tictac_board, False, alpha, beta, minmax_letter, si_agent_letter, val_success_for_letter, val_draw , depth+1,max_depth)
                tictac_board[possible_position] = ' '

                optimised_score = max(optimised_score, current_score)
                alpha = max(alpha, optimised_score)

                if alpha >= beta:
                    break

        return optimised_score

    else:
        optimised_score = math.inf
        for possible_position in tictac_board.keys():
            if tictac_board[possible_position] == ' ':
                tictac_board[possible_position] = si_agent_letter
                current_score = eval_minmax_with_alpha_beta_pruning(tictac_board, True, alpha, beta, minmax_letter, si_agent_letter, val_success_for_letter, val_draw,depth+1,max_depth)
                tictac_board[possible_position] = ' '

                optimised_score = min(optimised_score, current_score)
                beta = min(beta, optimised_score)

                if alpha >= beta:
                    break

        return optimised_score
    

def play_tic_tac_toe(tictac_board, si_agent_letter, minmax_letter, si_agent_plays_first):
    while True:
        if si_agent_plays_first:
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return "SIAgentWon"
            if val_draw(tictac_board):
                return "Draw"
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter)
            if val_success_for_letter(tictac_board, minmax_letter):
                return "MinMaxWon"
        else:
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter)
            if val_success_for_letter(tictac_board, minmax_letter):
                return "MinMaxWon"
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return "SIAgentWon"
            if val_draw(tictac_board):
                return "Draw"



def evaluate_tictac_board_state(tictac_board, current_letter, opponent_letter):
    score = 0
    lines = [
        (1, 2, 3), (4, 5, 6), (7, 8, 9),   # Horizontal
        (1, 4, 7), (2, 5, 8), (3, 6, 9),   # Vertical
        (1, 5, 9), (3, 5, 7)               # Diagonal
    ]

    # Score each line based on its contents
    for a, b, c in lines:
        line = [tictac_board[a], tictac_board[b], tictac_board[c]]
        if line.count(current_letter) == 2 and line.count(' ') == 1:
            score += 10  # Favorable position for 'current_letter'
        if line.count(opponent_letter) == 2 and line.count(' ') == 1:
            score -= 10  # Threatening position due to 'opponent_letter'

    return score


def evaluate_minmax_score(tictac_board, is_max_turn, current_letter, opponent_letter, depth, max_depth):
    if depth >= max_depth:  # Base case for maximum depth
        return evaluate_tictac_board_state(tictac_board, current_letter, opponent_letter)  

    if val_success_for_letter(tictac_board, current_letter):
        return 1 if is_max_turn else -1
    elif val_success_for_letter(tictac_board, opponent_letter):
        return -1 if is_max_turn else 1
    elif val_draw(tictac_board):
        return 0

    if is_max_turn:
        optimised_score = -math.inf
        for possible_position in tictac_board.keys():
            if tictac_board[possible_position] == ' ':
                tictac_board[possible_position] = current_letter
                current_score = evaluate_minmax_score(tictac_board, not is_max_turn, opponent_letter, current_letter, depth + 1, max_depth)
                tictac_board[possible_position] = ' '
                optimised_score = max(optimised_score, current_score)
        return optimised_score
    else:
        optimised_score = math.inf
        for possible_position in tictac_board.keys():
            if tictac_board[possible_position] == ' ':
                tictac_board[possible_position] = opponent_letter
                current_score = evaluate_minmax_score(tictac_board, not is_max_turn, opponent_letter, current_letter, depth + 1, max_depth)
                tictac_board[possible_position] = ' '
                optimised_score = min(optimised_score, current_score)
        return optimised_score

def min_max_move(tictac_board, minmax_letter, opponent_letter, max_depth):
    optimised_score = -math.inf
    optimised_position = None
    for possible_position in tictac_board.keys():
        if tictac_board[possible_position] == ' ':
            tictac_board[possible_position] = minmax_letter
            current_score = evaluate_minmax_score(tictac_board, True, minmax_letter, opponent_letter, 0, max_depth)
            tictac_board[possible_position] = ' '

            if current_score > optimised_score:
                optimised_score = current_score
                optimised_position = possible_position

    if optimised_position is not None:
        tictac_board[optimised_position] = minmax_letter
    return tictac_board



random starts, no pruning

In [4]:
import time
import random

def first_turn():
    return random.choice([1, 2])


def play_game_first_random_no_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = first_turn() == 1  # Randomly choose who plays first

    if SIAgent_plays_first:
        print("Semi-Intelligent Agent's Move:")
        tictac_board = si_agent_move(tictac_board, si_agent_letter)
        show_board(tictac_board)

    while True:
        if SIAgent_plays_first:
            print("MinMax Player's Move:")
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'

        else:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("MinMax Player's Move:")
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'


games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_first_random_no_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


Semi-Intelligent Agent's Move:


  |   |  
---------
  | X |  
---------
  |   |  

MinMax Player's Move:


O |   |  
---------
  | X |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   |  
---------
  | X |  
---------
  | X |  

MinMax Player's Move:


O | O |  
---------
  | X |  
---------
  | X |  

Semi-Intelligent Agent's Move:


O | O |  
---------
X | X |  
---------
  | X |  

MinMax Player's Move:


O | O | O
---------
X | X |  
---------
  | X |  

Semi-Intelligent Agent's Move:


  |   |  
---------
X |   |  
---------
  |   |  

MinMax Player's Move:


O |   |  
---------
X |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   | X
---------
X |   |  
---------
  |   |  

MinMax Player's Move:


O | O | X
---------
X |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | O | X
---------
X | X |  
---------
  |   |  

MinMax Player's Move:


O | O | X
---------
X | X | O
---------
  |   |  

Semi-Intelligent Agent's Move:


O | O | X
---

min max first, no aplha beta pruning

In [5]:
def play_game_first_min_max_no_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = False  # MinMax player goes first

    
    print("MinMax Player's Move:")
    tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
    show_board(tictac_board)

    
    SIAgent_plays_first = not SIAgent_plays_first

    while True:
        if SIAgent_plays_first:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'
        else:
            print("MinMax Player's Move:")
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

        SIAgent_plays_first = not SIAgent_plays_first  

# Simulation Loop
games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_first_min_max_no_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


MinMax Player's Move:


O |   |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   |  
---------
X |   |  
---------
  |   |  

MinMax Player's Move:


O | O |  
---------
X |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | O |  
---------
X |   |  
---------
  | X |  

MinMax Player's Move:


O | O | O
---------
X |   |  
---------
  | X |  

MinMax Player's Move:


O |   |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X |  
---------
  |   |  
---------
  |   |  

MinMax Player's Move:


O | X | O
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X | O
---------
  | X |  
---------
  |   |  

MinMax Player's Move:


O | X | O
---------
O | X |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X | O
---------
O | X | X
---------
  |   |  

MinMax Player's Move:


O | X | O
---------
O | X | X
---------
O |   |  

MinMax Player's Move:


O |   |  
---------
  |   |  
-

si agent first, no alpha beta pruning

In [6]:
import time
import random


def play_game_first_si_agent_no_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = True  # Semi-Intelligent Agent goes first

    
    print("Semi-Intelligent Agent's Move:")
    tictac_board = si_agent_move(tictac_board, si_agent_letter)
    show_board(tictac_board)

    while True:
        if SIAgent_plays_first:
            print("MinMax Player's Move:")
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'

        else:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("MinMax Player's Move:")
            tictac_board = min_max_move(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'


# Simulation Loop
games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_first_si_agent_no_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


Semi-Intelligent Agent's Move:


X |   |  
---------
  |   |  
---------
  |   |  

MinMax Player's Move:


X | O |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


X | O |  
---------
  |   |  
---------
X |   |  

MinMax Player's Move:


X | O | O
---------
  |   |  
---------
X |   |  

Semi-Intelligent Agent's Move:


X | O | O
---------
  |   |  
---------
X |   |  

MinMax Player's Move:


X | O | O
---------
O |   |  
---------
X |   |  

Semi-Intelligent Agent's Move:


X | O | O
---------
O |   | X
---------
X |   |  

MinMax Player's Move:


X | O | O
---------
O | O | X
---------
X |   |  

Semi-Intelligent Agent's Move:


X | O | O
---------
O | O | X
---------
X |   | X

MinMax Player's Move:


X | O | O
---------
O | O | X
---------
X | O | X

Semi-Intelligent Agent's Move:


  |   | X
---------
  |   |  
---------
  |   |  

MinMax Player's Move:


O |   | X
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   | X
---

random algo starts, with pruning

In [7]:
import random

def play_game_random_start_with_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = random.choice([True, False])  # Randomly decide who plays first

    if SIAgent_plays_first:
        print("Semi-Intelligent Agent Starts First.")
    else:
        print("MinMax Algorithm Starts First.")

    while True:
        if SIAgent_plays_first:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("MinMax Player's Move:")
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter,max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
        else:
            print("MinMax Player's Move:")
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter,max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
                
            if val_draw(tictac_board):
                return 'Draw'
            
# Simulation Loop
games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_random_start_with_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


Semi-Intelligent Agent Starts First.
Semi-Intelligent Agent's Move:


  | X |  
---------
  |   |  
---------
  |   |  

MinMax Player's Move:


O | X |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X |  
---------
  |   |  
---------
  | X |  

MinMax Player's Move:


O | X |  
---------
  | O |  
---------
  | X |  

Semi-Intelligent Agent's Move:


O | X |  
---------
  | O |  
---------
  | X | X

MinMax Player's Move:


O | X |  
---------
  | O |  
---------
O | X | X

Semi-Intelligent Agent's Move:


O | X | X
---------
  | O |  
---------
O | X | X

MinMax Player's Move:


O | X | X
---------
O | O |  
---------
O | X | X

MinMax Algorithm Starts First.
MinMax Player's Move:


O |   |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   |  
---------
  |   |  
---------
  |   | X

MinMax Player's Move:


O |   | O
---------
  |   |  
---------
  |   | X

Semi-Intelligent Agent's Move:


O | X | O
---------
  |   |  

min max first, with pruning

In [8]:
import random
import time
import math

def play_game_minmax_first_with_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = False  # MinMax player goes first

    
    print("MinMax Player's Move:")
    tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter, max_depth)
    show_board(tictac_board)

    
    SIAgent_plays_first = not SIAgent_plays_first

    while True:
        if SIAgent_plays_first:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'
        else:
            print("MinMax Player's Move:")
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

        
        SIAgent_plays_first = not SIAgent_plays_first

# Simulation Loop
games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_minmax_first_with_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


MinMax Player's Move:


O |   |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X |  
---------
  |   |  
---------
  |   |  

MinMax Player's Move:


O | X |  
---------
O |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X | X
---------
O |   |  
---------
  |   |  

MinMax Player's Move:


O | X | X
---------
O | O |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O | X | X
---------
O | O |  
---------
  |   | X

MinMax Player's Move:


O | X | X
---------
O | O | O
---------
  |   | X

MinMax Player's Move:


O |   |  
---------
  |   |  
---------
  |   |  

Semi-Intelligent Agent's Move:


O |   |  
---------
  |   |  
---------
  |   | X

MinMax Player's Move:


O |   | O
---------
  |   |  
---------
  |   | X

Semi-Intelligent Agent's Move:


O |   | O
---------
X |   |  
---------
  |   | X

MinMax Player's Move:


O | O | O
---------
X |   |  
---------
  |   | X

MinMax Player's Move:


O |   |  
---------
  |   |  
-

semi intelligent starts first, with pruning

In [9]:
def play_game_si_agent_first_with_pruning(max_depth=15):
    tictac_board, si_agent_letter, minmax_letter = original_board()
    SIAgent_plays_first = True  # Semi-Intelligent Agent starts first

    print("Semi-Intelligent Agent Starts First.")

    while True:
        if SIAgent_plays_first:
            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("MinMax Player's Move:")
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
        else:
            print("MinMax Player's Move:")
            tictac_board = min_max_move_with_alpha_beta_pruning(tictac_board, minmax_letter, si_agent_letter, max_depth)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, minmax_letter):
                return 'MinMaxWon'
            if val_draw(tictac_board):
                return 'Draw'

            print("Semi-Intelligent Agent's Move:")
            tictac_board = si_agent_move(tictac_board, si_agent_letter)
            show_board(tictac_board)
            if val_success_for_letter(tictac_board, si_agent_letter):
                return 'SIAgentWon'
                
            if val_draw(tictac_board):
                return 'Draw'

# Simulation Loop
games = 10
SIAgentWin = MinMaxWin = Draw = 0
startTime = time.time()

for _ in range(games):
    try:
        winner = play_game_si_agent_first_with_pruning()  
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"Error occurred: {e}")

totalTime = time.time() - startTime
print(f"Total Time: {totalTime} seconds")
print(f"SIAgent Wins: {SIAgentWin}, MinMax Wins: {MinMaxWin}, Draws: {Draw}")


Semi-Intelligent Agent Starts First.
Semi-Intelligent Agent's Move:


  |   |  
---------
  |   |  
---------
  | X |  

MinMax Player's Move:


  | O |  
---------
  |   |  
---------
  | X |  

Semi-Intelligent Agent's Move:


  | O |  
---------
  |   |  
---------
  | X | X

MinMax Player's Move:


  | O |  
---------
  |   |  
---------
O | X | X

Semi-Intelligent Agent's Move:


X | O |  
---------
  |   |  
---------
O | X | X

MinMax Player's Move:


X | O |  
---------
  | O |  
---------
O | X | X

Semi-Intelligent Agent's Move:


X | O |  
---------
X | O |  
---------
O | X | X

MinMax Player's Move:


X | O | O
---------
X | O |  
---------
O | X | X

Semi-Intelligent Agent Starts First.
Semi-Intelligent Agent's Move:


  |   |  
---------
  |   |  
---------
  | X |  

MinMax Player's Move:


  | O |  
---------
  |   |  
---------
  | X |  

Semi-Intelligent Agent's Move:


  | O |  
---------
  |   |  
---------
X | X |  

MinMax Player's Move:


  | O |  
---------
  |